In [22]:
import pandas as pd
df = pd.read_csv("parlaspeech.csv")
df.columns

Index(['file', 'start', 'end', 'words', 'word_start_times', 'word_end_times',
       'asr', 'true_file', 'reconame', 'longname', 'yt_hash', 'hashname',
       'key', 'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',
       'split'],
      dtype='object')

In [23]:
df_pruned = df.loc[:, ['file', 'start', 'end',
     'true_file', 'reconame', 'yt_hash', 'hashname',
     'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',]]

In [24]:
df_pruned.to_csv("parlaspeech_pruned.csv", index=False)

In [25]:
df_pruned.to_json("parlaspeech_pruned.json", orient="records", lines=True)

In [26]:
c_to_drop = df_pruned.was_in_samples | df_pruned.sim<0.8
df_pruned = df_pruned.drop(index=df_pruned.index[c_to_drop])
available = 224031104 
corpus_size = 1316725396

train_fraction = available/corpus_size
train_fraction

0.17014261643359388

Zaokrožim na 0.15

In [27]:
keep_df = df_pruned.sample(frac=0.15, random_state=42)
keep_df.to_csv("keep_pruned.csv", index=False)

In [ ]:
from shutil import copyfile
import os
for file in keep_df.hashname:
    if os.path.exists(os.path.join("/home/rupnik/macocu/task8/transfer", file)):
        continue
    copyfile(os.path.join("/home/rupnik/macocu/task8/data", file),
            os.path.join("/home/rupnik/macocu/task8/transfer", file)
            )

In [30]:
from concurrent.futures import ProcessPoolExecutor

def copy(file):
    if os.path.exists(os.path.join("/home/rupnik/macocu/task8/transfer", file)):
        return
    copyfile(os.path.join("/home/rupnik/macocu/task8/data", file),
            os.path.join("/home/rupnik/macocu/task8/transfer", file)
            )

with ProcessPoolExecutor(max_workers=10) as executor:
    executor.map(copy, keep_df.hashname.values)